In [14]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


from animal_shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
#DONE# change for your username and password and CRUD Python module name
username = "aacuser"
password = "KyHa0722"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Module 6 Assignment')

image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design


app.layout = html.Div([
    #html.Div(id='hidden-div', style={'display':'none'}),
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image', height= 100, width=80),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard - Kyle Hake'))),
    html.Div([
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 1},
                {'label': 'Mountain Rescue', 'value': 2},
                {'label': 'Disaster Rescue', 'value': 3},
                {'label': 'Reset', 'value': 4}
            ],
            value=0,
            labelStyle = {'display': 'inline-block'},
     
        )
    ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        row_deletable=False,
        selected_columns="single",
        selected_rows="single",
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
     html.Hr(),
     html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]



@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])

def update_dashboard(filter_type): 
    df = pd.DataFrame.from_records(animals.read({}))
    
    if ('value'==1):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog",
        "breed":{"$in":["Labrador Retriever Mix","Chesa Bay Retr Mix","Newfoundland Mix"]},
        "sex_upon_outcome":"Intact Female",
        "age_upon_outcome_in_weeks":{"$gte":"26"},"age_upon_outcome_in_weeks":{"$lte":"156"}})))
    elif ('value'==2):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog",
        "breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":"26"},"age_upon_outcome_in_weeks":{"$lte":"156"}})))
    elif ('value'==3):
        df = pd.DataFrame(list(shelter.read({"animal_type":"Dog",
        "breed":{"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
        "sex_upon_outcome":"Intact Male",
        "age_upon_outcome_in_weeks":{"$gte":"20"},"age_upon_outcome_in_weeks":{"$lte":"300"}})))
    elif ('value' == 4):
        df = pd.DataFrame.from_records(animals.read({}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    
        
    return (data, columns)

@app.callback(
    Output("graph-id", "children"), 
    [Input('datatable-id', "data")])

def generate_chart(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
            figure = px.pie(dff, values='breed', names='breed', color_discrete_sequence=px.colors.sequential.RdBu)
        )
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data")])

def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("breed"),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]
#FIXME Add in the code for your geolocation chart


app